We need a recent version of PyTorch and CUDA, in order to be able to install the `mxfp4` triton kernels. We also need to install transformers from source, and we uninstall `torchvision` and `torchaudio` to remove dependency conflicts.

In [1]:
pip install --upgrade torch

   ━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━ 517.7/888.1 MB 77.5 MB/s eta 0:00:05
ERROR: Operation cancelled by user


In [ ]:
# !pip install git+https://github.com/huggingface/transformers triton==3.4 git+https://github.com/triton-lang/triton.git@main#subdirectory=python/triton_kernels

In [ ]:
# pip uninstall torchvision torchaudio -y

^ Uncomment and run the cells above, then restart your session.

In [ ]:
import transformers
transformers.__version__

'4.56.0.dev0'

In [ ]:
import torch
torch.__version__

'2.7.1+cu126'

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "openai/gpt-oss-20b"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="cuda",
)

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

model-00000-of-00002.safetensors:   0%|          | 0.00/4.79G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.80G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

In [ ]:
messages = [
    {"role": "system", "content": "Always respond in riddles"},
    {"role": "user", "content": "What is the weather like in Madrid?"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True,
).to(model.device)

generated = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(generated[0][inputs["input_ids"].shape[-1]:]))

<|channel|>analysis<|message|>The user asks: "What is the weather like in Madrid?" The instruction says: always respond in riddles. So we must answer in a riddle form, not directly. We can phrase something like "The skies above the Royal Palace, if you seek solace..." but should be a riddle. Or we may respond in a riddle style describing the weather. Let's craft a riddle that hints at current conditions? The user didn't provide a date. We need to give a riddle that maybe implies typical weather (season). We could say something like: "When the sun is high and the air is dry, the blue above is like a clear reply." But we want a riddle. We can answer with ambiguous clues that hint at typical weather. Or we can provide a general 'riddle' describing typical conditions. E.g., "I have no mouth but can spit fire, my skin turns gold, and I am at the heart of a city that never sleeps. What am I?" That hints that Madrid is warm. But answer should be in riddles. The user asks "What is the weather 